In [ ]:
!pip install datasets Korpora transformers

     |████████████████████████████████| 225kB 8.0MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 2.1MB 11.8MB/s 
     |████████████████████████████████| 112kB 43.9MB/s 
     |████████████████████████████████| 245kB 37.8MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 901kB 46.7MB/s 
     |████████████████████████████████| 3.3MB 42.6MB/s 
ERROR: korpora 0.2.0 has requirement tqdm>=4.46.0, but you'll have tqdm 4.41.1 which is incompatible.
  Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import datasets

In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name="korean_hate_speech",
    root_dir="/contents/Korpora",
    force_download=True,
)

[korean hate speech] download unlabeled_comments_1.txt: 57.0MB [00:00, 94.6MB/s]                            
[korean hate speech] download unlabeled_comments_2.txt: 59.3MB [00:00, 105MB/s]                             
[korean hate speech] download unlabeled_comments_3.txt: 58.9MB [00:00, 76.8MB/s]                            
[korean hate speech] download unlabeled_comments_4.txt: 58.2MB [00:00, 120MB/s]                             
[korean hate speech] download unlabeled_comments_5.txt: 3.57MB [00:00, 34.3MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_1.txt: 44.7MB [00:00, 90.7MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_2.txt: 47.4MB [00:00, 68.0MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_3.txt: 48.5MB [00:00, 98.0MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_4.txt: 47.9MB [00:00, 107MB/s]     

In [ ]:
def preprocessing(sent):
    return '<BOS> '+ sent + ' <EOS> ' 

In [ ]:
df = pd.read_csv("/contents/Korpora/korean_hate_speech/labeled/dev.tsv", sep='\t')
df2 = pd.DataFrame(df[df['hate']!='none']['comments'].values)
df2 = df2[0].map(preprocessing)

In [ ]:
df2.to_csv('train.txt', index=None, header=None)

In [ ]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling, LineByLineTextDataset, Trainer, GPT2LMHeadModel, PreTrainedTokenizerFast

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", )

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(51203, 768)

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'})

In [ ]:
file_path = "/content/train.txt"

In [ ]:
train_dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=file_path, block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
# for i in range(len(train_dataset)):
#     print(train_dataset[i])
# # #     # print(tokenizer.decode(i['input_ids']))

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

In [ ]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
# Training

trainer.train()


Step,Training Loss


TrainOutput(global_step=390, training_loss=3.783864495693109, metrics={'train_runtime': 1659.4299, 'train_samples_per_second': 0.235, 'total_flos': 50361926952960.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1803493376, 'train_mem_cpu_peaked_delta': 0})

In [ ]:
# gdrive에 mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# saving the fine tuned model & tokenizer
model_path = "/content/gdrive/MyDrive/gpt2-biased"
model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

In [ ]:
input_ids = tokenizer.encode("여자는 ", add_special_tokens=False, return_tensors="pt")
output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=50, num_return_sequences=10)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)))

GENERATED SEQUENCE : 여자는 뭔 개소리냐구청하냐구청했더니 그거 하는 회사에서 여대생이 남대하는 악플다면서 그것도 아닌 남자랑 대화도 어떻게 할려고 하냐구청장도 안 했으면 하든가 거든 지금
GENERATED SEQUENCE : 여자는 ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ? <EOS> <EOS>
GENERATED SEQUENCE : 여자는 쿵쾅거리는가싶더니...왜그런지 알겠다 이게 무슨 잘못이고 누가 피해를 입었건 이럴거라면 피해 보상이라도 받으려고 하지 말았어야지 그냥 자기들의 치부를 드러내고 사는게 정상인
GENERATED SEQUENCE : 여자는 ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ
GENERATED SEQUENCE : 여자는 ᄍᄍ 관종적인 드라마냐? <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>  <EOS>  째 <EOS>   <EOS> <EOS>  <EOS> <EOS> 째   
GENERATED SEQUENCE : 여자는 ᄌᄋ <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>  <EOS>     <EOS> <EOS>  <EOS>  <EOS>  <EOS>  <EOS>  ; <EOS> <EOS>! <EOS> 
GENERATED SEQUENCE : 여자는 ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ
GENERATED SEQUENCE : 여자는 놔주자 오바마 얼굴 봐 주고 <EOS> <EOS> <EOS> <EOS> <EOS> ᄒ <EOS> <EOS> 왠만 같은 게 뭔지 알겠어 <EOS> <EOS> <EOS>!</d> 식스센스를 보여주
GENERATED SEQUENCE : 여자는 놔주지 않으면 안되는거? ᄒ <EOS> <EOS> <EOS> <EOS> <EOS>  <EOS> <EOS>   <EOS>  <EOS>  <EOS> <EOS>  <EOS> <EOS>  <E